In [1]:
import joblib
import keras
import numpy as np
from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from typing import Literal

from romgw.config.env import PROJECT_ROOT
from romgw.typing.core import (
    BBHSpinType,
    ModeType,
)

2025-12-04 00:43:48.071935: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-04 00:43:48.266531: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-04 00:43:52.108334: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
def working_model_files(
    bbh_spin: BBHSpinType,
    mode: ModeType,
    model_name: Literal["NonLinRegV1"],
) -> dict[str, Path]:
    """"""
    mode_dir = PROJECT_ROOT / "data" / bbh_spin / "train" / mode

    param_scaler = (mode_dir / "amplitude" / "models" / model_name /
                    "x_scaler.gz")
    
    model = lambda c: mode_dir / c / "models" / model_name / "model.keras"
    waveform_scaler = lambda c: (mode_dir / c / "models" / model_name /
                                 "y_scaler.gz")
    interpolation_matrix = lambda c: (mode_dir / c /
                                      "empirical_interpolation" /
                                      "B_matrix.npy")
    
    return {"param_scaler": param_scaler,
            "amp_model": model("amplitude"),
            "amp_scaler": waveform_scaler("amplitude"),
            "amp_interpolation_matrix": interpolation_matrix("amplitude"),
            "phi_model": model("phase"),
            "phi_scaler": waveform_scaler("phase"),
            "phi_interpolation_matrix": interpolation_matrix("phase")}

In [3]:
def final_model_files(
    bbh_spin: BBHSpinType,
    mode: ModeType,
    model_name: Literal["NonLinRegV1"],
) -> dict[str, Path]:
    """"""
    param_scaler = PROJECT_ROOT / "models" / "x_scaler.gz"
    
    model_dir = PROJECT_ROOT / "models" / bbh_spin / model_name / mode

    model = lambda c: model_dir / c / "model.keras"
    waveform_scaler = lambda c: model_dir / c / "y_scaler.gz"
    interpolation_matrix = lambda c: model_dir / c / "interpolation_matrix.npy"

    return {"param_scaler": param_scaler,
            "amp_model": model("amplitude"),
            "amp_scaler": waveform_scaler("amplitude"),
            "amp_interpolation_matrix": interpolation_matrix("amplitude"),
            "phi_model": model("phase"),
            "phi_scaler": waveform_scaler("phase"),
            "phi_interpolation_matrix": interpolation_matrix("phase")}

In [4]:
def finalise_model_files(
    working_model_files: dict[str, Path],
    final_model_files: dict[str, Path],
) -> None:
    """"""
    for file in final_model_files.values():
        file.parent.mkdir(parents=True, exist_ok=True)

    # param scaler
    param_scaler = joblib.load(working_model_files["param_scaler"])
    joblib.dump(param_scaler, final_model_files["param_scaler"])

    # models
    amp_model = keras.models.load_model(working_model_files["amp_model"])
    amp_model.save(final_model_files["amp_model"])
 
    phi_model = keras.models.load_model(working_model_files["phi_model"])
    phi_model.save(final_model_files["phi_model"])

    # waveform scalers
    amp_scaler = joblib.load(working_model_files["amp_scaler"])
    joblib.dump(amp_scaler, final_model_files["amp_scaler"])

    phi_scaler = joblib.load(working_model_files["phi_scaler"])
    joblib.dump(phi_scaler, final_model_files["phi_scaler"])

    # interpolation matrices
    amp_interpolation_matrix = np.load(
        file=working_model_files["amp_interpolation_matrix"],
        allow_pickle=False
    )
    np.save(final_model_files["amp_interpolation_matrix"],
            amp_interpolation_matrix,
            allow_pickle=False)

    phi_interpolation_matrix = np.load(
        file=working_model_files["phi_interpolation_matrix"],
        allow_pickle=False
    )
    np.save(final_model_files["phi_interpolation_matrix"],
            phi_interpolation_matrix,
            allow_pickle=False)

In [5]:
bbh_spin = "NS"
mode = "2,2"
model_name = "NonLinRegV1"

In [6]:
working_files = working_model_files(bbh_spin, mode, model_name)
print(f"{working_files=}")
for f in working_files.values():
    print(f)
print("")
final_files = final_model_files(bbh_spin, mode, model_name)
print(f"{final_files=}")
for f in final_files.values():
    print(f)

working_files={'param_scaler': PosixPath('/home/connor/projects/reduced-order-modelling-of-gravitational-waves/data/NS/train/2,2/amplitude/models/NonLinRegV1/x_scaler.gz'), 'amp_model': PosixPath('/home/connor/projects/reduced-order-modelling-of-gravitational-waves/data/NS/train/2,2/amplitude/models/NonLinRegV1/model.keras'), 'amp_scaler': PosixPath('/home/connor/projects/reduced-order-modelling-of-gravitational-waves/data/NS/train/2,2/amplitude/models/NonLinRegV1/y_scaler.gz'), 'amp_interpolation_matrix': PosixPath('/home/connor/projects/reduced-order-modelling-of-gravitational-waves/data/NS/train/2,2/amplitude/empirical_interpolation/B_matrix.npy'), 'phi_model': PosixPath('/home/connor/projects/reduced-order-modelling-of-gravitational-waves/data/NS/train/2,2/phase/models/NonLinRegV1/model.keras'), 'phi_scaler': PosixPath('/home/connor/projects/reduced-order-modelling-of-gravitational-waves/data/NS/train/2,2/phase/models/NonLinRegV1/y_scaler.gz'), 'phi_interpolation_matrix': PosixPath

In [7]:
finalise_model_files(working_files, final_files)

2025-12-04 00:43:54.921545: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
